In [1]:
import os
import re

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext,load_index_from_storage

In [2]:
# 设置llm模型
llm_name="qwen2.5:latest"
embedding_name="quentinz/bge-large-zh-v1.5:latest"
base_url='http://localhost:11434'

Settings.llm = Ollama(
    model=llm_name, 
    request_timeout=360.0,
    base_url=base_url)

# 设置embedding model 
Settings.embed_model = OllamaEmbedding(
    model_name=embedding_name,
    base_url=base_url)

In [ ]:
import glob
from copy import deepcopy
from llama_index.core.schema import Document,TextNode

def extract_images_under_headers(md_content):
    headers = re.findall(r'## (.*?)\n+(.*?)(?=\n##|\Z)', md_content, re.DOTALL)
    
    images_under_headers = {}
    for header, content in headers:
        images = re.findall(r'!\[.*?\]\((.*?)\)', content)
        images_under_headers[header] = images
    return images_under_headers

documents=[]
md_files=glob.glob('./preprocess/*.md')
for md_file in md_files:
    with open(md_file,encoding='utf-8') as fr:
        md_content='\n'.join(fr.readlines())
    page_num=int(os.path.basename(md_file).split('-')[0][3:])
    documents.append((page_num,md_content))

def get_page_nodes(docs,separator="\n---\n"):
    nodes=[]
    for idx,doc in docs:
        doc_chunks=doc.split(separator)
        for doc_chunk in doc_chunks:
            images_under_headers=extract_images_under_headers(doc_chunk)

            filter_header_images={}
            for header in images_under_headers:
                if len(images_under_headers[header])>0:
                    filter_header_images[header]=images_under_headers[header]
            metadata={'file_path':md_file,'page_number':idx}
            if len(filter_header_images)>0:
                metadata.update(filter_header_images)
                # 去掉内容中的图片文本
                pattern = r"!{1}\[.*\)"
                doc_chunk= re.sub(pattern, "\n", doc_chunk)
            else:
                pass

            node=TextNode(
                text=doc_chunk,
                metadata=deepcopy(metadata),
            )
        nodes.append(node)
    return nodes


nodes=get_page_nodes(documents)
print(nodes[12].metadata)

{'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 118, '通过空调辅助功能菜单调节': ['images\\img117-125.png'], '设置前排座椅通风时间': ['images\\img118-126.png']}


In [4]:
from llama_index.postprocessor.xinference_rerank import XinferenceRerank
from llama_index.core.postprocessor import LLMRerank

# node_postprocessors=[XinferenceRerank(top_n=5,model='bge-reranker-large',base_url='http://192.168.3.155:9997')]
node_postprocessors=[LLMRerank(top_n=10)]

In [5]:
from pydantic import BaseModel, Field
from typing import List


class Output(BaseModel):
    """Output containing the response, page numbers, images, and confidence."""

    response: str = Field(..., description="The answer to the question.")
    page_numbers: List[int] = Field(
        ...,
        description="The page numbers of the sources used to answer this question. Do not include a page number if the context is irrelevant.",
    )
    page_images: List[str] = Field(...,description="The images used to answer this question, given only image name, do not format using markdown")
    confidence: float = Field(
        ...,
        description="Confidence value between 0-1 of the correctness of the result.",
    )
    confidence_explanation: str = Field(
        ..., description="Explanation for the confidence score"
    )
    
llm = Ollama(
    model=llm_name, 
    request_timeout=360.0,
    base_url=base_url)

sllm = llm.as_structured_llm(output_cls=Output)

In [6]:
import glob
from llama_index.core.schema import TextNode

data_path='./preprocess/'
vertordb_dir='vector_storage'

if os.path.exists(vertordb_dir):
    storage_context=StorageContext.from_defaults(persist_dir=vertordb_dir)
    index=load_index_from_storage(storage_context=storage_context)
else:
    # documents = SimpleDirectoryReader(data_path,required_exts=[".md"]).load_data(show_progress=True)
    # index=VectorStoreIndex.from_documents(documents=documents,show_progress=True)

    index=VectorStoreIndex(nodes=nodes,show_progress=True)
    index.storage_context.persist(persist_dir=vertordb_dir)

query_engine= index.as_query_engine(
    similarity_top_k=5,
    llm=sllm,
    # node_postprocessors=node_postprocessors,
    # response_mode="compact" # you can also select other modes like `compact`, `refine`
    )

c:\Users\wushaogui\miniconda3\envs\langchian\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 223/223 [00:09<00:00, 22.34it/s]


In [7]:
import math
from IPython.display import Markdown

def generate_markdown_table(image_paths):
    num_images = len(image_paths)
    if num_images==0:
        return ''
    
    num_cols = max(1,math.ceil(math.sqrt(num_images)))
    num_rows = math.ceil(num_images / num_cols)

    table = "| " + " | ".join(["-"] * num_cols) + " |\n"
    table += "| " + " | ".join(["---"] * num_cols) + " |\n"

    for row in range(num_rows):
        row_images = image_paths[row * num_cols:(row + 1) * num_cols]
        table += "| " + " | ".join([f"![Image](./preprocess/{img})" for img in row_images])
        table += " |" + " |" * (num_cols - len(row_images)) + "\n"

    return table

def show_response(response):
    md_content=''
    md_content+=generate_markdown_table(response.response.page_images)

    # 添加文本
    md_content+='\n---\n'
    md_content+=response.response.response

    display(Markdown(md_content))

In [8]:
query_str='如何从锁定状态唤醒中央显示器?'
response = query_engine.query(query_str)
show_response(response)
response.source_nodes

| - | - |
| --- | --- |
| ![Image](./preprocess/images\img261-301.png) | ![Image](./preprocess/images\img262-302.png) |

---
您可以通过长按中控台按钮约5秒进入锁屏模式，在这种状态下，点击中央显示屏的任意位置或再次长按中控台按钮可以重新解锁屏幕。

[NodeWithScore(node=TextNode(id_='f6c007f0-c495-4ae4-a6c4-d0282b8e0db9', embedding=None, metadata={'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 264, '调节中央显示屏显示模式': ['images\\img263-305.png'], '调节中央显示屏亮度': ['images\\img264-306.png']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 设置中央显示屏显示状态\n\n## 调节中央显示屏显示模式\n\n在中央显示屏中点击\n\n-设置-显示，进入显示设置界面。\n\n\n01 点击设置中央显示屏显示模式（日间模式、夜间模式、自\n\n动）。\n\n说明！\n\n您可以依据个人喜好选择自动模式：\n\n□日出到日落：白天显示日间模式，晚上显示夜间模式。\n\n□自定时段：依据设置的时间段切换显示模式。\n\n□日夜模式选择自动模式后，中央显示屏会自动切换日间模式\n\n或夜间模式。\n\n## 调节中央显示屏亮度\n\n在中央显示屏中点击\n\n-设置-显示，进入显示设置界面。\n\n\n01 点击设置中央显示屏亮暗模式。\n\n02滑动滑条调节中央显示屏亮度。\n\n您还可以通过以下方式调节中央显示屏亮度：\n\n■中央显示屏车辆功能界面，请参见车辆功能界面（页码266）。\n\n■仪表板上的调光器，请参见调节背光亮度（页码91）。\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.5298833886721772),
 NodeWithScore(node=TextNod

In [9]:
query_str='后雨刮和洗涤器?'
response = query_engine.query(query_str)
show_response(response)
response.source_nodes


---
后雨刮和洗涤器的操作方式如下：

1. 将拨杆沿远离方向盘的方向推动，启动后挡风玻璃雨刮和洗涤器。
2. 按压拨杆末端上方区域，启动后挡风玻璃间歇性刮刷模式。
3. 按压拨杆末端下方区域，启动后挡风玻璃连续刮刷模式。

注意事项包括：
- 冬季使用雨刮前，请先清除挡风玻璃上的冰和积雪并确认雨刮片没有冻结在挡风玻璃上。
- 当挡风玻璃上有尘沙、鸟粪、昆虫、树浆等异物时，请先清洁挡风玻璃，否则会损坏雨刮片/影响雨刮片清洁效果。
- 避免在挡风玻璃干燥的情况下开启雨刮，否则可能导致雨刮片和挡风玻璃损坏。
- 定期清洁和检查雨刮片，否则雨刮片使用寿命可能会缩短。
- 应使用合格的洗涤液，不合格的洗涤液可能导致洗涤器损坏。
- 如果发现雨刮片橡胶硬化或有裂纹、雨刮片在挡风玻璃上留下划痕或不能刮洗某个区域，则需要更换雨刮片。

[NodeWithScore(node=TextNode(id_='9995f4a9-0aa2-477c-b077-712228aedd03', embedding=None, metadata={'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 66, '倒车时后挡风玻璃自动刮刷': ['images\\img66-63.png']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 后雨刮和洗涤器\n\n\n将拨杆沿远离方向盘的方向推动，启动后挡风玻璃雨刮和洗涤器。\n\n按压拨杆末端上方区域，启动后挡风玻璃间歇性刮刷模\n\n式。\n\n按压拨杆末端下方区域，启动后挡风玻璃连续刮刷模式。\n\n注意！\n\n■冬季使用雨刮前，请先清除挡风玻璃上的冰和积雪并确认雨刮片\n\n没有冻结在挡风玻璃上。\n\n■当挡风玻璃上有尘沙、鸟粪、昆虫、树浆等异物时，请先清洁挡\n\n风玻璃，否则会损坏雨刮片/影响雨刮片清洁效果。\n\n■避免在挡风玻璃干燥的情况下开启雨刮，否则可能导致雨刮片和\n\n挡风玻璃损坏。\n\n■定期清洁和检查雨刮片，否则雨刮片使用寿命可能会缩短。\n\n■应使用合格的洗涤液，不合格的洗涤液可能导致洗涤器损坏。\n\n■如果发现雨刮片橡胶硬化或有裂纹、雨刮片在挡风玻璃上留下划\n\n痕或不能刮洗某个区域，则需要更换雨刮片。\n\n## 倒车时后挡风玻璃自动刮刷\n\n前雨刮启用且换挡杆拨至倒挡（R）后，后雨刮会自动开启。\n\n在中央显示屏中点击\n\n-车辆设置-雨刮，进入雨刮设置界面。\n\n\n01 点击开启/关闭后雨刮倒车联动功能。\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0

In [10]:
query_str='安全带如何清洁?'
response = query_engine.query(query_str)
show_response(response)
response.source_nodes


---
使用安全带专用清洁剂和海绵来清洁安全带。请将安全带置于阴凉处彻底干燥后再使用。切勿使用漂白剂、染料或化学溶剂清洁安全带，这些材料可能严重影响安全带的织物性能。

[NodeWithScore(node=TextNode(id_='b78c4b5e-7b24-4891-b137-dfeea5ba1df6', embedding=None, metadata={'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 102, '系紧安全带': ['images\\img101-97.png', 'images\\img101-98.png'], '松开安全带': ['images\\img102-99.png'], '孕期安全': ['images\\img102-100.png']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 使用安全带\n\n## 系紧安全带\n\n1 调节座椅到合适位置。\n\n2 缓慢拉出安全带，将锁舌插入锁扣中，直到听见“咔哒”声。\n\n3\n\n使腰部安全带应尽可能低的横跨于胯部。\n\n确保肩部安全带斜跨整个肩部，穿过胸部。\n\n\n4 将前排座椅安全带高度调整至合适的位置。\n\n\n捏住调节机构，上下调节安全带高度至合适位置。\n\n警告！\n\n■请勿将座椅靠背太过向后倾斜。\n\n■请在系紧安全带前检查锁扣插口是否存在异物（如：食物残渣\n\n等），若存在异物请及时取出。\n\n■为确保安全带正常工作，请务必将安全带插入与之匹配的锁扣\n\n中。\n\n■乘坐时，安全带必须拉紧，防止松垮，并确保其牢固贴身，无扭\n\n曲。\n\n■切勿将安全带从您的后背绕过、从您的胳膊下面绕过或绕过您的\n\n颈部。安全带应远离您的面部和颈部，但不得从肩部滑落。\n\n■如果安全带无法正常使用，请联系Lynk & Co领克中心进行处理。\n\n## 松开安全带\n\n\n安全带锁扣解锁按钮\n\n按下安全带锁扣上的解锁按钮，可以让安全带自行缩回。\n\n如果安全带未完全缩回，手动使其缩回，不松散悬垂。\n\n警告\n\n■松开安全带时，应在按下解锁按钮之前握住安全带，以防止安全\

In [11]:
query_str='组合仪表内容介绍？'
response = query_engine.query(query_str)
show_response(response)
response.source_nodes

| - | - | - |
| --- | --- | --- |
| ![Image](./preprocess/images\img69-64.png) | ![Image](./preprocess/images\img78-65.png) | ![Image](./preprocess/images\img79-66.png) |
| ![Image](./preprocess/images\img79-67.png) | ![Image](./preprocess/images\img80-68.png) | ![Image](./preprocess/images\img80-69.png) |
| ![Image](./preprocess/images\img81-70.png) | ![Image](./preprocess/images\img81-71.png) | ![Image](./preprocess/images\img82-72.png) |

---
组合仪表包含多个显示区域和功能，具体包括：

1. 左侧信息显示区域 - 显示当前车速、驾驶员辅助系统等相关信息。
2. 挡位指示图标 - 显示车辆当前挡位。
3. 右侧信息显示区域 - 显示多媒体、电话、胎压、车辆行程等信息，并会根据车辆状态弹出相应的报警提示信息。
4. 纯电可续航里程 - 显示仅依靠动力电池剩余电量可行驶的距离。
5. 动力电池电量 - 显示动力电池电量。
6. 能量回收等级 - 显示当前能量回收等级（自动、低、中、高）。
7. 驾驶模式 - 显示当前车辆的驾驶模式。
8. 燃油量表 - 显示油箱中剩余的燃油油位。
9. 剩余燃油可续航里程 - 显示剩余燃油可行驶的距离。

此外，组合仪表还提供了多种功能设置，例如通过调光旋钮调节背光亮度、查看和重置行程信息等。

[NodeWithScore(node=TextNode(id_='88e694c2-0817-499d-9738-2820473df774', embedding=None, metadata={'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 69}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 组合仪表\n\n![](images\\img69-64.png)01 左侧信息显示区域：显示车辆当前车速、驾驶员辅助系统等相\n\n关信息。\n\n02挡位指示图标：显示车辆当前挡位。\n\n03右侧信息显示区域：显示多媒体、电话、胎压、车辆行程信息\n\n等，同时会根据车辆状态弹出相应的报警提示信息。\n\n04纯电可续航里程：显示仅依靠动力电池剩余电量可行驶的距\n\n离。\n\n05动力电池电量：显示动力电池电量。\n\n06能量回收等级：显示车辆当前能量回收等级（自动、低、中、\n\n高）。\n\n07驾驶模式：显示当前车辆的驾驶模式。\n\n08燃油量表：显示油箱中剩余的燃油油位。\n\n09剩余燃油可续航里程：显示剩余燃油可行驶的距离。\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.507996335838344),
 NodeWithScore(node=TextNode(id_='64ea7654-9b94-401c-87a0-6c3165d95388', embedding=None, metadata={'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 80, '机舱盖/车门/尾门/安全带提醒界面': ['i

In [12]:
query_str='什么是遥控泊车（RPA）？'
response = query_engine.query(query_str)
show_response(response)
response.source_nodes


---
遥控泊车（RPA）是一种通过Lynk & Co App，在车外一定范围内控制车辆进行泊入、泊出、直线前进或后退的远程泊车功能。它主要包括以下几种功能：直线遥控功能，遥控泊入功能以及遥控泊出功能。使用此功能前需要阅读并同意免责声明，并且作为驾驶员您应遵守相关法律要求并对安全停车承担全部责任。

[NodeWithScore(node=TextNode(id_='f1695fb9-f448-439b-a200-2ad8dc3b256c', embedding=None, metadata={'file_path': './preprocess\\doc99-安全出行-73.md', 'page_number': 236, '直线遥控功能': ['images\\img233-261.png'], '遥控泊入功能': ['images\\img234-262.png'], '遥控泊出功能': ['images\\img234-263.png', 'images\\img235-264.png', 'images\\img235-265.png']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 遥控泊车\n\n遥控泊车（RPA）是一种远程泊车控制功能，可在车外一定范围内使\n\n用Lynk & Co App控制车辆进行泊入、泊出、直线前进或后退。\n\nRPA主要包含以下功能：\n\n■直线遥控功能。\n\n■遥控泊入功能。\n\n■遥控泊出功能。\n\n说明！\n\n□通过Lynk & Co App使用RPA功能前需要阅读并同意免责声明，因\n\n此请认真阅读免责声明，确保在已知晓并接受相关条件或条款的\n\n情况下开启并使RPA功能。\n\n□遥控泊车辅助在泊车入位过程中会代替驾驶员执行转向和制动的\n\n操作，但驾驶员仍需注意观察周围环境，必要时结束泊车过程，\n\n以避免某些障碍物因系统无法识别而影响行车安全。\n\n□作为驾驶员，您应遵守相关法律要求，并对安全停车承担全部责\n\n任。\n\nRPA可支持三种车位：水平车位、垂直车位、斜列车位。\n\n\n说明\n\n□RPA工作时车辆会自动执行转向和制动操作。\n\n## 直线遥控功能\n\n启用RPA功能后，可以按照以下步骤控制车辆直线行驶：\n\n1 启用RPA：在Lynk & Co App主界面中点击\n\n功能按键，开启RPA\n\n功能